Summary： 5/26/2019

objective: extract 家常菜

status: all information is there. 


In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
import urllib.request
from bs4 import BeautifulSoup

import pandas as pd
import re

In [2]:
### This function only extracts "家常菜" dishes which has 10 pages on this website:https://www.xiachufang.com/category/40076/
### To do: get the hyperref to the actual recipe for each dish

root_dir = 'https://www.xiachufang.com/'

def extract():
    res = []
    
    for i in range(1,2):
        url = 'https://www.xiachufang.com/category/40076/?page=' + str(i)       
        resp = urllib.request.urlopen(url)
        soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))
    
        dishs = soup.find_all('div',class_="recipe recipe-215-horizontal pure-g image-link display-block")
        #print("dishs:",dishs)
        
        for dish in dishs:
            # dish name (I did not do much cleanning on the name)
            name = dish.find('p',class_="name").find('a').text.strip()
            
            # dish ingredients 
            ingredients_temp = dish.find('p',class_="ing ellipsis").text.strip().split('、')
            #print("ingredients_temp:",ingredients_temp)
            
            # ingredients cleanning: remove the content in (), e.g. 香醋(保宁醋/镇江香醋))
            ingredients = []
            for ing in ingredients_temp:
                if '(' in ing:
                    ing = ing.split('(')[0]
                elif '（' in ing:
                    ing = ing.split('（')[0]
                
                ingredients.append(ing)
            
            # used to extract rating and num_ppl_made
            stats = dish.find('p',class_="stats").text.strip().splitlines()
            
            # 综合评分, I don't know how this number/rating is calculated.
            try:
                rating = float(stats[0].replace(u'\xa0', ' ').strip().split(' ')[1])
            except:
                rating= 0
            
            # how many people have made this dish in the past seven days
            try:
                num_ppl_made = int(stats[1].replace(u'\xa0', ' ').strip().split(' ')[1])
            except:
                num_ppl_made = 0
                
            # hyperlinks which contain refers to recipes    
            hyperlinks = set([link.get('href') for link in dish.find_all('a')])
            recipe_link = ''
            for link in hyperlinks:
                if 'recipe' in link:
                    recipe_link = link
                    
            recipe_link = root_dir + recipe_link
            
            #instructions
            recipe_resp = urllib.request.urlopen(recipe_link)
            recipe_soup = BeautifulSoup(recipe_resp, from_encoding=resp.info().get_param('charset'))
            instructions = recipe_soup.find_all(property="og:description")[0]['content']
         
            # images
            imagelink = dish.find('img')['data-src']
            
            res.append((name, ingredients, rating, num_ppl_made,hyperlinks,imagelink,instructions))
    return pd.DataFrame(res,columns=['name', 'ingredients', 'rating', 'num_ppl_made','hyperlinks','imagelink','instructions'])

In [3]:
# with this data, we can make queries. E.g. which recipe includes "胡萝卜"

# it takes a few minutes to scrape all the data

recipes = extract()

In [4]:
recipes.head(2)

,name,ingredients,rating,num_ppl_made,hyperlinks,imagelink,instructions
0,可乐鸡翅,"[鸡翅, 可乐, 盐, 料酒, 老抽, 生抽, 生姜片, 桂皮]",8.2,75,"{/cook/10001401/, /category/2362/, /category/2...",http://i1.chuimg.com/ed35e338873811e6b87c0242a...,1.在生鸡翅上划刀，或者用牙签给每个鸡翅正反面共戳6个小洞，这样可以更加入味。用一勺老抽5m...
1,懒人版糖醋排骨,"[猪小排, 料酒, 盐, 酱油, 米醋, 白糖, 姜片]",8.5,53,"{/category/1563/, /category/1525/, /category/1...",http://i2.chuimg.com/4daad8ea877a11e6a9a10242a...,1.猪小排冼净，晾干水份备用；2.锅内倒少量油，烧热之后，爆香姜片；3.放入排骨，一直煸炒到...


In [5]:
recipes.to_csv('recipes.csv')

In [44]:
# example test
url = 'https://www.xiachufang.com/' + 'recipe/1000229'       
resp = urllib.request.urlopen(url)
soup = BeautifulSoup(resp, from_encoding=resp.info().get_param('charset'))

instructions = soup.find_all(property="og:description")[0]['content']
instructions

'1.在生鸡翅上划刀，或者用牙签给每个鸡翅正反面共戳6个小洞，这样可以更加入味。用一勺老抽5ml，两勺料酒10ml腌至少两小时 【时间紧可省略腌制，但要划刀和戳洞，否则不易入味】；2.锅中放冷水和葱姜片，鸡翅与冷水一同下锅煮开，不用煮透，看到浮沫比较多时关火捞出即可，用温热的水一个个冲洗掉浮沫，沥干；3.用厨房用纸擦干，有杂毛可以这时候拔干净；4.【最好用不粘锅】把锅用姜片涂一遍可以防止鸡翅粘锅；5...'

In [ ]:
### the other links are for the image attached to each step of instructions

In [58]:
soup

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="lt-ie9 lt-ie8 lt-ie7 show-anti-ie"> <![endif]--><!--[if IE 7]>    <html class="lt-ie9 lt-ie8 show-anti-ie"> <![endif]--><!--[if IE 8]>    <html xmlns:ng="http://angularjs.org" class="lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class=""> <!--<![endif]-->
<head>
<title>
【可乐鸡翅的做法步骤图，可乐鸡翅怎么做好吃】渍_下厨房</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8;" http-equiv="content-type"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//s.chuimg.com" rel="dns-prefetch"/>
<meta content="width=device-width" name="viewport"/>
<meta content="app-id=460979760" name="apple-itunes-app"/>
<meta content="HNAbHIxmbo" name="baidu-site-verification"/>
<meta content="webkit" name="renderer"/>
<link href="https://s.chuimg.com/favicon.ico" rel="shortcut icon"/>
<script src="//dup.baidustatic.com/js/ds.js"></script>
<link href="https://www.xiachufang.com/recip

In [19]:
### a simple query

has_鸡翅 = recipes['ingredients'].apply(lambda x: '鸡翅' in x)

recipes[has_鸡翅]

,name,ingredients,rating,num_ppl_made,hyperlinks,imagelink
0,可乐鸡翅,"[鸡翅, 可乐, 盐, 料酒, 老抽, 生抽, 生姜片, 桂皮]",8.2,65,"{/category/2406/, /category/3905/, /recipe/100...",http://i1.chuimg.com/ed35e338873811e6b87c0242a...
9,可乐鸡翅,"[鸡翅, 干红辣椒, 可乐, 老抽]",8.1,10,"{/category/2406/, /category/3905/, /cook/10009...",http://i2.chuimg.com/59735f92873b11e6a9a10242a...
18,蜜汁鸡翅,"[鸡翅, 蒜, 姜, 料酒, 生抽, 蜂蜜]",7.9,8,"{/cook/101621549/, /category/2110/, /recipe/10...",http://i2.chuimg.com/f6bb3d5e885511e6b87c0242a...


In [11]:
# try to extend to any query

# query = ['鸡翅']


# def search(query, ls):
#     return set(query) <= set(ls)

# result = recipes[recipes['ingredients'].apply(lambda x: True if '鸡翅' in x)]